In [0]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [0]:
! nvidia-smi

Sun Mar 22 19:23:12 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

# Import; installation

In [0]:
import os
import sys
!pip install gdown
import gdown

from pathlib import Path

In [0]:
mmdetection_repo = 'https://github.com/open-mmlab/mmdetection'
if not os.path.exists('mmdetection/'):
    !git clone -q {mmdetection_repo}
    # gdown.download(url_pikachu_dataset, output_pikachu_dataset, False)
    # gdown.download(url_init, output_init, False)
    # gdown.download(url_config, output_config, False)
    !cd mmdetection/ && pip install -q -r requirements.txt
    !cd mmdetection/ && python setup.py develop 
    sys.path.append(os.path.abspath('mmdetection/'))

Streaming output truncated to the last 5000 lines.
   T * data() const {
 ^ ~~
mmdet/ops/sigmoid_focal_loss/src/sigmoid_focal_loss_cuda.cu:160:423: warning: ‘T* at::Tensor::data() const [with T = long int]’ is deprecated [-Wdeprecated-declarations]
   AT_DISPATCH_FLOATING_TYPES_AND_HALF(
                                                                                                                                                                                                                                                                                                                                                                                                                                       ^
/usr/local/lib/python3.6/dist-packages/torch/include/ATen/core/TensorBody.h:322:1: note: declared here
   T * data() const {
 ^ ~~
mmdet/ops/sigmoid_focal_loss/src/sigmoid_focal_loss_cuda.cu:160:467: warning: ‘T* at::Tensor::data() const [with T = double]’ is deprecated [-Wdeprecated-d

In [0]:
!pip install --upgrade Pillow

     |████████████████████████████████| 2.1MB 4.9MB/s 
ERROR: mmdet 1.1.0+e03c96d has requirement Pillow<=6.2.2, but you'll have pillow 7.0.0 which is incompatible.
  Found existing installation: Pillow 6.2.2
    Uninstalling Pillow-6.2.2:
      Successfully uninstalled Pillow-6.2.2


In [0]:
from __future__ import division
import argparse
import os

import torch
from mmcv import Config

from mmdetection.mmdet import __version__
from mmdet.apis import (get_root_logger, set_random_seed, # TODO: init_dist
                        train_detector)
from mmdet.datasets import build_dataset
from mmdet.models import build_detector

# Data loading

In [0]:
labels_url = 'https://raw.githubusercontent.com/morganmcg1/stanford-cars/master/labels_df.csv'
!wget {labels_url} -P {output_data}

--2020-03-22 20:30:30--  https://raw.githubusercontent.com/morganmcg1/stanford-cars/master/labels_df.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1372076 (1.3M) [text/plain]
Saving to: ‘data/stanford/labels_df.csv’

labels_df.csv       100%[===================>]   1.31M  --.-KB/s    in 0.06s   

2020-03-22 20:30:31 (21.2 MB/s) - ‘data/stanford/labels_df.csv’ saved [1372076/1372076]



In [0]:
url_data = 'http://imagenet.stanford.edu/internal/car196/cars_train.tgz'
train_file = 'data/stanford/cars_train.tgz'

data_main = Path('./data/')
output_data = data_main / 'stanford'

!mkdir -p "./{output_data}"

#images
!wget {url_data} -P {output_data}
!tar zxvf {train_file} {output_data}
!tar zxvf {train_file} -C {output_data}

#labels
labels_url = 'https://raw.githubusercontent.com/morganmcg1/stanford-cars/master/labels_df.csv'
!wget {labels_url} -P {output_data}

--2020-03-22 19:50:17--  http://imagenet.stanford.edu/internal/car196/cars_train.tgz
Resolving imagenet.stanford.edu (imagenet.stanford.edu)... 171.64.68.16
Connecting to imagenet.stanford.edu (imagenet.stanford.edu)|171.64.68.16|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 979269282 (934M) [application/x-gzip]
Saving to: ‘data/stanford/cars_train.tgz’

cars_train.tgz      100%[===================>] 933.90M  67.5MB/s    in 13s     

2020-03-22 19:50:30 (69.8 MB/s) - ‘data/stanford/cars_train.tgz’ saved [979269282/979269282]



# Configuration

In [0]:
import pandas as pd
import json

In [0]:
cfg = Config.fromfile('mmdetection/configs/fcos/fcos_center_r50_caffe_fpn_gn_1x_4gpu.py.py')

cfg['data']['imgs_per_gpu'] = 64
cfg['data']['workers_per_gpu'] = 1

In [0]:
distributed = False # niestety w Colabie nie będzie distributed
logger = get_root_logger(cfg.log_level)
logger.info('Distributed training: {}'.format(distributed))

2020-03-22 20:01:30,391 - mmdet - INFO - Distributed training: False


In [0]:
model = build_detector(cfg.model, train_cfg=cfg.train_cfg, test_cfg=cfg.test_cfg)

2020-03-22 20:01:41,081 - mmdet - INFO - load model from: open-mmlab://resnet50_caffe
Downloading: "https://s3.ap-northeast-2.amazonaws.com/open-mmlab/pretrain/third_party/resnet50_caffe-788b5fa3.pth" to /root/.cache/torch/checkpoints/resnet50_caffe-788b5fa3.pth


2020-03-22 20:01:47,014 - mmdet - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: conv1.bias, layer1.0.conv1.bias, layer1.0.conv2.bias, layer1.0.conv3.bias, layer1.0.downsample.0.bias, layer1.1.conv1.bias, layer1.1.conv2.bias, layer1.1.conv3.bias, layer1.2.conv1.bias, layer1.2.conv2.bias, layer1.2.conv3.bias, layer2.0.conv1.bias, layer2.0.conv2.bias, layer2.0.conv3.bias, layer2.0.downsample.0.bias, layer2.1.conv1.bias, layer2.1.conv2.bias, layer2.1.conv3.bias, layer2.2.conv1.bias, layer2.2.conv2.bias, layer2.2.conv3.bias, layer2.3.conv1.bias, layer2.3.conv2.bias, layer2.3.conv3.bias, layer3.0.conv1.bias, layer3.0.conv2.bias, layer3.0.conv3.bias, layer3.0.downsample.0.bias, layer3.1.conv1.bias, layer3.1.conv2.bias, layer3.1.conv3.bias, layer3.2.conv1.bias, layer3.2.conv2.bias, layer3.2.conv3.bias, layer3.3.conv1.bias, layer3.3.conv2.bias, layer3.3.conv3.bias, layer3.4.conv1.bias, layer3.4.conv2.bias, layer3.4.conv3.bias, layer3.5.conv

In [0]:
labels = pd.read_csv(output_data/'labels_df.csv')
labels.head()
labels.tail()

,filename,bbox_x1,bbox_y1,bbox_x2,bbox_y2,class_id,class_name,is_test,filename_cropped,bbox_h,bbox_w
0,00001.jpg,39,116,569,375,14,Audi TTS Coupe 2012,0,cropped_00001.jpg,260,531
1,00002.jpg,36,116,868,587,3,Acura TL Sedan 2012,0,cropped_00002.jpg,472,833
2,00003.jpg,85,109,601,381,91,Dodge Dakota Club Cab 2007,0,cropped_00003.jpg,273,517
3,00004.jpg,621,393,1484,1096,134,Hyundai Sonata Hybrid Sedan 2012,0,cropped_00004.jpg,704,864
4,00005.jpg,14,36,133,99,106,Ford F-450 Super Duty Crew Cab 2012,0,cropped_00005.jpg,64,120


,filename,bbox_x1,bbox_y1,bbox_x2,bbox_y2,class_id,class_name,is_test,filename_cropped,bbox_h,bbox_w
16180,test_08037.jpg,49,57,1169,669,63,Chevrolet Sonic Sedan 2012,1,test_08037.jpg,613,1121
16181,test_08038.jpg,23,18,640,459,16,Audi V8 Sedan 1994,1,test_08038.jpg,442,618
16182,test_08039.jpg,33,27,602,252,17,Audi 100 Sedan 1994,1,test_08039.jpg,226,570
16183,test_08040.jpg,33,142,521,376,38,BMW Z4 Convertible 2012,1,test_08040.jpg,235,489
16184,test_08041.jpg,77,73,506,380,32,BMW X5 SUV 2007,1,test_08041.jpg,308,430


trzeba rozdzielic te kategorie z iteracji i zrobic uniqua po 2 kolumnach; 

chyba trzeba nappisac klase /content/mmdetection/mmdet/datasets/coco.py dla tego datesetu

In [0]:
def build_annotation_json_from_frame(frame):
  images = []
  annotations = []
  for _, row in frame.head(2).iterrows():
    im_id = row.filename.split('.')[0]
    images.append({
        "file_name": row.filename,
        "id": im_id,
    })
    annotations.append({
        'image_id':im_id,
        'id':im_id,
        'bbox':[row.bbox_x1, row.bbox_y1, row.bbox_x2, row.bbox_y2],
        'category_id': row.class_id,
    })
  categories = list(frame[['class_id', 'class_name']]
                .groupby(['class_id', 'class_name'])
                .size()
                .reset_index()
                .drop(0, axis=1)
                .rename({'class_id': 'id'}, axis=1)
                .to_dict('index')
                .values())
  return {
      'images': images,
      'annotations': annotations,
      'categories': categories
  }

In [88]:
annotations_json_file = output_data / "annotations.json"
with open(annotations_json_file, "w") as p: 
     p.write(json.dumps(build_annotation_json_from_frame(labels)))

11528

In [0]:
data_root = output_data
dataset_type = 'CocoDataset'
cfg.data_root = data_root
img_norm_cfg = dict(
    mean=[102.9801, 115.9465, 122.7717], std=[1.0, 1.0, 1.0], to_rgb=False)


train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations', with_bbox=True),
    dict(type='Resize', img_scale=(1333, 800), keep_ratio=True),
    dict(type='RandomFlip', flip_ratio=0.5),
    dict(type='Normalize', **img_norm_cfg),
    dict(type='Pad', size_divisor=32),
    dict(type='DefaultFormatBundle'),
    dict(type='Collect', keys=['img', 'gt_bboxes', 'gt_labels']),
]
test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(
        type='MultiScaleFlipAug',
        img_scale=(1333, 800),
        flip=False,
        transforms=[
            dict(type='Resize', keep_ratio=True),
            dict(type='RandomFlip'),
            dict(type='Normalize', **img_norm_cfg),
            dict(type='Pad', size_divisor=32),
            dict(type='ImageToTensor', keys=['img']),
            dict(type='Collect', keys=['img']),
        ])
]

data = dict(
    imgs_per_gpu=1,
    workers_per_gpu=1,
    train=dict(
        type=dataset_type,
        ann_file=annotations_json_file,
        img_prefix=data_root,
        pipeline=train_pipeline),
    test=dict(
        type=dataset_type,
        ann_file=annotations_json_file,
        img_prefix=data_root,
        pipeline=test_pipeline),
    val=dict(
        type=dataset_type,
        ann_file=annotations_json_file,
        img_prefix=data_root,
        pipeline=test_pipeline))

cfg['data'] = data

In [92]:
datasets = [build_dataset(cfg.data.train)]

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


KeyError: ignored

In [0]:
zbudowac dataset